## Examples of queries that can be added to the set of examples for few shot learning

In [ ]:
# Which actors have worked with Keanu Reeves in more than 2 movies and list the movies
MATCH (p1:Person)-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(p2:Person)
WHERE p1.name = "Keanu Reeves" AND p1.name <> p2.name
WITH p1, p2, collect(m.title) AS Movies, COUNT(m) AS sharedMovies
WHERE sharedMovies > 2
RETURN p2.name AS Actor, Movies
ORDER BY sharedMovies DESC

# Fetch movie recommendations in the same genre that have Keanu Reeves
MATCH (p:Person {{name: "Keanu Reeves"}})-[:ACTED_IN]->(m:Movie),
(m)-[:IN_CATEGORY]->(c:Category),
(m2:Movie)-[:IN_CATEGORY]->(c)
WHERE NOT (p)-[:ACTED_IN]->(m2) AND m <> m2
RETURN DISTINCT m2.title AS RecommendedMovies, collect(c.name) AS Genres
LIMIT 10;
MATCH (p:Person {{name: "Keanu Reeves"}})-[:ACTED_IN]->(m:Movie),
(m)-[:IN_CATEGORY]->(c:Category),
(m2:Movie)-[:IN_CATEGORY]->(c)
WHERE NOT (p)-[:ACTED_IN]->(m2) AND m <> m2
RETURN DISTINCT m2.title AS RecommendedMovies, collect(c.name) AS Genres
LIMIT 10

# Find people who collaborated  with Christopher Nolan and list them in alphabetical order
MATCH (p:Person {{name: "Christopher Nolan"}})-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person)
RETURN DISTINCT a.name AS Actor
ORDER BY a.name;

# Find movies that share at least one genre with Stranger Things and released within 2 years that Stranger Things was released
MATCH (movie:Movie {{title: "Stranger Things"}})-[:IN_CATEGORY]->(genre:Category),
(movie)-[:CREATED_ON]->(year:Year)
WITH movie, collect(genre) AS genres, year
MATCH (otherMovie:Movie)-[:IN_CATEGORY]->(otherGenre:Category),
(otherMovie)-[:CREATED_ON]->(otherYear:Year)
WHERE movie <> otherMovie
WITH movie, genres, year, otherMovie, otherYear, collect(otherGenre) AS otherGenres
WHERE any(g IN genres WHERE g IN otherGenres) AND
abs(year.value - otherYear.value) <= 2
RETURN otherMovie.title AS Recommendation,
[g IN otherGenres | g.name] AS SharedGenres,
otherYear.value AS ReleaseYear
LIMIT 10;